In [5]:
import os
import time
from PIL import Image, ImageDraw, ImageFont
import osmnx as ox
import matplotlib.pyplot as plt
from collections import defaultdict

# City dataset
city_data = [
    {"name": "Brasília, Brazil", "coords": (-15.793889, -47.882778), "dist": 2000, "group": "Radial Power", "taxonomy": "Centralized Burst"},
    {"name": "Palmanova, Italy", "coords": (45.9061, 13.3095), "dist": 1000, "group": "Radial Power", "taxonomy": "Radial Convergence"},
    {"name": "Versailles, France", "coords": (48.8049, 2.1204), "dist": 1600, "group": "Radial Power", "taxonomy": "Radial Implosion"},

    {"name": "Karlsruhe, Germany", "coords": (49.0069, 8.4037), "dist": 1500, "group": "Layered Hierarchies", "taxonomy": "Centralized Ring"},
    {"name": "Canberra, Australia", "coords": (-35.2811, 149.1287), "dist": 1800, "group": "Layered Hierarchies", "taxonomy": "Circled Globe"},
    {"name": "Washington, D.C., USA", "coords": (38.8899, -77.0091), "dist": 1800, "group": "Layered Hierarchies", "taxonomy": "Segmented Radial Convergence"},

    {"name": "Ciudad Lineal, Spain", "coords": (40.4395, -3.6540), "dist": 1200, "group": "Linear Flows", "taxonomy": "Flow Chart"},
    {"name": "Manhattan, New York, USA", "coords": (40.7549, -73.9840), "dist": 1300, "group": "Linear Flows", "taxonomy": "Arc Diagram"},
    {"name": "Chicago, USA", "coords": (41.8781, -87.6300), "dist": 1700, "group": "Linear Flows", "taxonomy": "Elliptical Implosion"},

    {"name": "Greater London, UK", "coords": (51.5072, -0.1276), "dist": 2000, "group": "Modular Networks", "taxonomy": "Area Grouping"},
    {"name": "Amsterdam, Netherlands", "coords": (52.3738, 4.8910), "dist": 1100, "group": "Modular Networks", "taxonomy": "Ramification"},
    {"name": "Paris, France", "coords": (48.8566, 2.3522), "dist": 1500, "group": "Modular Networks", "taxonomy": "Scaling Circles"},

    {"name": "Fez, Morocco", "coords": (34.065, -4.973), "dist": 1000, "group": "Emergent Forms", "taxonomy": "Organic Rhizome"},
    {"name": "Old Delhi, India", "coords": (28.6562, 77.2410), "dist": 1200, "group": "Emergent Forms", "taxonomy": "Circular Ties"},
    {"name": "Athens, Greece", "coords": (37.9755, 23.7348), "dist": 1300, "group": "Emergent Forms", "taxonomy": "Sphere"}
]

# Folders
os.makedirs("images", exist_ok=True)
os.makedirs("comparisons", exist_ok=True)

# OSMnx settings
ox.settings.use_cache = True
ox.settings.overpass_endpoint = "https://overpass.kumi.systems/api"

# Generate maps
successful_maps = 0
for entry in city_data:
    city = entry["name"]
    coords = entry["coords"]
    dist = entry["dist"]
    filename = city.replace(', ', '_').replace(' ', '_') + ".png"
    filepath = os.path.join("images", filename)

    try:
        print(f"🗺️  Generating map for {city}...")
        G = ox.graph_from_point(coords, dist=dist, network_type="drive")
        G_proj = ox.project_graph(G)
        ox.plot_graph(
            G_proj,
            bgcolor="white",
            node_color="black",
            node_size=0,
            edge_color="black",
            edge_linewidth=0.3,
            show=False,
            save=True,
            filepath=filepath,
            dpi=300
        )
        plt.close("all")  # suppress notebook display
        time.sleep(1)  # throttle API
        successful_maps += 1
    except Exception as e:
        print(f"⚠️  Failed to process {city} with coords={coords} and dist={dist}: {e}")

# Fonts
font_size = 20
title_font_size = 26
try:
    font = ImageFont.truetype("arial.ttf", font_size)
    title_font = ImageFont.truetype("arial.ttf", title_font_size)
except:
    font = ImageFont.load_default()
    title_font = ImageFont.load_default()

# Layout
thumb_size = (600, 600)
panel_width = 2 * thumb_size[0] + 3 * 40
panel_height = thumb_size[1] + 3 * font_size + 80

# Group cities
grouped = defaultdict(list)
for entry in city_data:
    grouped[entry["group"]].append(entry)

# Generate panels
comparison_images = []
successful_panels = 0

for group_name in sorted(grouped.keys()):
    # Title page
    title_page = Image.new("RGB", (panel_width, panel_height), "white")
    draw = ImageDraw.Draw(title_page)
    draw.text(
        (panel_width // 2 - draw.textlength(group_name, font=title_font) // 2, panel_height // 2),
        group_name, font=title_font, fill="black"
    )
    comparison_images.append(title_page)

    for entry in grouped[group_name]:
        taxonomy_label = entry["taxonomy"]
        city_name = entry["name"]
        image_name = city_name.replace(', ', '_').replace(' ', '_') + ".png"
        tax_file = os.path.join("taxonomy", f"{taxonomy_label}.jpg")
        city_file = os.path.join("images", image_name)

        if not os.path.exists(tax_file) or not os.path.exists(city_file):
            print(f"❌ Missing file for {taxonomy_label} or {city_name}")
            continue

        tax_img = Image.open(tax_file).convert("RGB").resize(thumb_size)
        city_img = Image.open(city_file).convert("RGB").resize(thumb_size)

        panel = Image.new("RGB", (panel_width, panel_height), "white")
        draw = ImageDraw.Draw(panel)
        x1, x2 = 40, thumb_size[0] + 2 * 40
        draw.text((panel_width // 2 - draw.textlength(group_name, font=title_font) // 2, 20),
                  group_name, font=title_font, fill="black")
        panel.paste(tax_img, (x1, 60))
        panel.paste(city_img, (x2, 60))
        draw.text((x1, 60 + thumb_size[1] + 10), taxonomy_label, font=font, fill="black")
        draw.text((x2, 60 + thumb_size[1] + 10), city_name, font=font, fill="black")

        comparison_images.append(panel)
        successful_panels += 1
        print(f"✅ Panel for {taxonomy_label} vs {city_name}")

# Export PDF
if comparison_images:
    pdf_output = "comparison.pdf"
    comparison_images[0].save(
        pdf_output,
        save_all=True,
        append_images=comparison_images[1:],
        format="PDF"
    )
    print(f"\n📄 PDF exported to: {pdf_output}")
else:
    print("⚠️ No images to compile into PDF.")

# Summary
print(f"\n✅ {successful_maps} maps generated.")
print(f"🖼️  {successful_panels} comparison panels created.")
print("🎯 Done.")

🗺️  Generating map for Brasília, Brazil...
🗺️  Generating map for Palmanova, Italy...
🗺️  Generating map for Versailles, France...
🗺️  Generating map for Karlsruhe, Germany...
🗺️  Generating map for Canberra, Australia...
🗺️  Generating map for Washington, D.C., USA...
🗺️  Generating map for Ciudad Lineal, Spain...
🗺️  Generating map for Manhattan, New York, USA...
🗺️  Generating map for Chicago, USA...
🗺️  Generating map for Greater London, UK...
🗺️  Generating map for Amsterdam, Netherlands...
🗺️  Generating map for Paris, France...
🗺️  Generating map for Fez, Morocco...
🗺️  Generating map for Old Delhi, India...
🗺️  Generating map for Athens, Greece...
✅ Panel for Organic Rhizome vs Fez, Morocco
✅ Panel for Circular Ties vs Old Delhi, India
✅ Panel for Sphere vs Athens, Greece
✅ Panel for Centralized Ring vs Karlsruhe, Germany
✅ Panel for Circled Globe vs Canberra, Australia
✅ Panel for Segmented Radial Convergence vs Washington, D.C., USA
✅ Panel for Flow Chart vs Ciudad Lineal, Sp